In [ ]:
import ray 

ray.init(num_gpus=1)

In [ ]:
import ray
from ray.rllib.agents.ppo import PPOTrainer
config = {
    "env": "LunarLander-v2",
    "num_gpus":1,
}
stop = {"episode_reward_mean": 250}
ray.shutdown()
ray.init(
  num_gpus=1,
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)
# execute training 
analysis = ray.tune.run(
  "PPO",
  config=config,
  stop=stop,
  checkpoint_at_end=True,
  local_dir="LunarLander_v2"  
)

In [ ]:
# restore a trainer from the last checkpoint
trial = analysis.get_best_logdir("episode_reward_mean", "max")
checkpoint = analysis.get_best_checkpoint(
  trial,
  "training_iteration",
  "max",
)
trainer = PPOTrainer(config=config)
trainer.restore(checkpoint)

In [ ]:
import gym
from gym.wrappers import RecordVideo

env = RecordVideo(gym.make("LunarLander-v2"),"ppo_video")
obs = env.reset()

while True:
    action = trainer.compute_action(obs)
    obs,reward,done, _ = env.step(action)
    if done:
        break
env.close()